Link to dataset: [Here](https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Pretty display for notebooks
%matplotlib inline

# Load data and cleanup

In [ ]:
df_intakes = pd.read_csv('./data/aac_intakes.csv')
df_intakes_outcomes = pd.read_csv('./data/aac_intakes_outcomes.csv')
df_outcomes = pd.read_csv('./data/aac_outcomes.csv')

## Check dataframe shapes

In [ ]:
print("Intakes:\n\tLines: {}\n\tColumns: {}".format(df_intakes.shape[0], df_intakes.shape[1]))
print("Intakes_outcomes:\n\tLines: {}\n\tColumns: {}".format(df_intakes_outcomes.shape[0], df_intakes_outcomes.shape[1]))
print("Outcomes:\n\tLines: {}\n\tColumns: {}".format(df_outcomes.shape[0], df_outcomes.shape[1]))

## Check head content

### Intakes

In [ ]:
df_intakes.head(5)

## Intakes_outcomes

In [ ]:
df_intakes_outcomes.head(4)

### Outcomes

In [ ]:
df_outcomes.head(4)

### Clean data

In [ ]:
# Clean intakes
df_intakes.drop('datetime2', axis=1, inplace=True)
df_intakes.drop('found_location', axis=1, inplace=True)

# Clean intakes_outcomes
df_intakes_outcomes.drop('intake_month', axis=1, inplace=True)
df_intakes_outcomes.drop('intake_year', axis=1, inplace=True)
df_intakes_outcomes.drop('intake_monthyear', axis=1, inplace=True)
df_intakes_outcomes.drop('intake_weekday', axis=1, inplace=True)
df_intakes_outcomes.drop('intake_hour', axis=1, inplace=True)
df_intakes_outcomes.drop('time_in_shelter', axis=1, inplace=True)   #time_in_shelter_days is more usable

# Clean outcomes
df_outcomes.drop('monthyear', axis=1, inplace=True)
